# Importing Libraries

In [215]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [216]:
# Get the url from MoneyControl

url = 'https://www.moneycontrol.com/stocks/fno/marketstats/futures/active_value/homebody.php?opttopic=active_value&optinst=stkfut&sel_mth=1&sort_order=0'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="https://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Most Active Value - F&amp;O Market, All Futures &amp; Options, Index Futures, Index Options, Stock Futures, Stock Options</title>
<meta content="View Most Active Value in F&amp;O Market Action by All Futures, All Options, Index Futures, Index Options, Stock Futures, Stock Options filter by All Expiries &amp; Expiries for a particular date" name="description"/>
<!-- Moneycontrol Common header -> Start -->
<script type="text/javascript">
			var max_limit_interstitial_page_check = 8;
			var interstitial_page_position = [1, 3, 5, 7];
							function load_vignette_solution_script_file() {
					let head = document.querySelector( 'head' );
					let script = document.createElement('script');
					script.id = 'ads-vignette-solution';
					script.s

In [217]:
# Finding table from the html code 

soup.find('table', class_ = 'tblList')

<table class="tblList" width="100%">
<form id="fno_pnc_frm" method="post" name="fno_pnc_frm">
<input id="instrument_type" name="instrument_type" type="hidden"/>
<input id="sc_id" name="sc_id" type="hidden"/>
<input id="ind_id" name="ind_id" type="hidden"/>
<input id="option_type" name="option_type" type="hidden"/>
<input id="user_sel_price" name="user_sel_price" type="hidden"/>
<input id="sel_exp_date" name="sel_exp_date" type="hidden"/>
<input id="post_flag" name="post_flag" type="hidden"/>
</form>
<tr>
<th width="16%">Symbol </th>
<th width="8%">Expiry<br>Date</br></th>
<th width="6%">Last<br>Price</br></th>
<th width="5%">Change</th>
<th width="7%">Chge<br/>%</th>
<th width="7%">High<br/>Low</th>
<th width="7%">Average<br/>Price</th>
<th width="10%">Vol - Shares<br/>Contracts </th>
<th width="8%">Value<br/>(Rs. Lakh)</th>
<th width="10%">Open<br/>Interest</th>
<th width="7%">Open Int Chg</th>
</tr>
<tr>
<td class="TAL"><a class="bl_12" href="/india/fnoquote/tatapower/TPC/2024-03-28/

In [218]:
# Fetching the first table since it has the information that we need

table = soup.find_all('table')[1]

In [219]:
# Fetching column names. 'th' here represents table headers
column_name = table.find_all('th')
column_name

[<th width="16%">Symbol </th>,
 <th width="8%">Expiry<br>Date</br></th>,
 <th width="6%">Last<br>Price</br></th>,
 <th width="5%">Change</th>,
 <th width="7%">Chge<br/>%</th>,
 <th width="7%">High<br/>Low</th>,
 <th width="7%">Average<br/>Price</th>,
 <th width="10%">Vol - Shares<br/>Contracts </th>,
 <th width="8%">Value<br/>(Rs. Lakh)</th>,
 <th width="10%">Open<br/>Interest</th>,
 <th width="7%">Open Int Chg</th>]

In [220]:
column_name = [header.text.strip() for header in column_name]

print(column_name)

['Symbol', 'ExpiryDate', 'LastPrice', 'Change', 'Chge%', 'HighLow', 'AveragePrice', 'Vol - SharesContracts', 'Value(Rs. Lakh)', 'OpenInterest', 'Open Int Chg']


In [221]:
df = pd.DataFrame(columns = column_name)
df

,Symbol,ExpiryDate,LastPrice,Change,Chge%,HighLow,AveragePrice,Vol - SharesContracts,Value(Rs. Lakh),OpenInterest,Open Int Chg


In [222]:
# Fetching column rows. 'tr' here represents table rows.

column_data = table.find_all('tr')
column_data

[<tr>
 <th width="16%">Symbol </th>
 <th width="8%">Expiry<br>Date</br></th>
 <th width="6%">Last<br>Price</br></th>
 <th width="5%">Change</th>
 <th width="7%">Chge<br/>%</th>
 <th width="7%">High<br/>Low</th>
 <th width="7%">Average<br/>Price</th>
 <th width="10%">Vol - Shares<br/>Contracts </th>
 <th width="8%">Value<br/>(Rs. Lakh)</th>
 <th width="10%">Open<br/>Interest</th>
 <th width="7%">Open Int Chg</th>
 </tr>,
 <tr>
 <td class="TAL"><a class="bl_12" href="/india/fnoquote/tatapower/TPC/2024-03-28/FUTSTK/XX/0.00/true" title="TATAPOWER"><b>TATAPOWER</b></a></td>
 <td>28-Mar-24</td>
 <td>430.15</td>
 <td><span class="gr_11" style="color:#0F6C02">33.10</span></td>
 <td><span class="gr_11" style="color:#0F6C02">8.34%</span></td>
 <td>435.60<br/>
 								396.25</td>
 <td>420.68</td>
 <td>181,905,750<br/>
 								53,898</td>
 <td>765,241.11</td>
 <td>87,122,250</td>
 <td><span class="gr_11" style="color:#0F6C02">10,881,000<br/>
 14.27%</span></td>
 </tr>,
 <tr class="odd">
 <td cla

### Filling out rows 

We are iterating over each item inside our html parsed list and saving it in our dataframe. Also, we are removing unnecessary indentation using the replace() function.

In [223]:
for row in column_data[1:]:
    row_data = row.find_all('td')
    individual_row_data = [i.text.strip() for i in row_data]
    final_data = [i.replace('\r\n\t\t\t\t\t\t\t\t', '- ') for i in individual_row_data]
    final_data_v2 = [i.replace('\r\n', ': ') for i in final_data]
    print(final_data_v2)
    length = len(df)
    df.loc[length] = final_data_v2
    

['TATAPOWER', '28-Mar-24', '430.15', '33.10', '8.34%', '435.60- 396.25', '420.68', '181,905,750- 53,898', '765,241.11', '87,122,250', '10,881,000: 14.27%']
['TATACHEM', '28-Mar-24', '1,292.70', '119.00', '10.14%', '1,338.30- 1,168.00', '1,273.72', '44,781,000- 81,420', '570,384.55', '14,716,350', '1,632,950: 12.48%']
['TATASTEEL', '28-Mar-24', '157.80', '5.20', '3.41%', '159.85- 153.00', '157.19', '262,174,000- 47,668', '412,111.31', '234,762,000', '23,589,500: 11.17%']
['TATAMOTORS', '28-Mar-24', '1,039.50', '15.90', '1.55%', '1,048.90- 1,016.50', '1,036.78', '39,627,825- 27,809', '410,853.36', '57,753,825', '-1,239,750: -2.10%']
['BEL', '28-Mar-24', '215.90', '4.40', '2.08%', '217.75- 212.40', '215.26', '115,385,100- 20,243', '248,377.97', '138,441,600', '-2,707,500: -1.92%']
['HDFCBANK', '28-Mar-24', '1,449.30', '1.30', '0.09%', '1,455.10- 1,445.25', '1,449.43', '16,569,850- 30,127', '240,168.38', '180,727,800', '-4,179,450: -2.26%']
['SBIN', '28-Mar-24', '788.10', '1.35', '0.17%', 

# Final table

In [224]:
df.head(20)

,Symbol,ExpiryDate,LastPrice,Change,Chge%,HighLow,AveragePrice,Vol - SharesContracts,Value(Rs. Lakh),OpenInterest,Open Int Chg
0,TATAPOWER,28-Mar-24,430.15,33.10,8.34%,435.60- 396.25,420.68,"181,905,750- 53,898","765,241.11","87,122,250","10,881,000: 14.27%"
1,TATACHEM,28-Mar-24,"1,292.70",119.00,10.14%,"1,338.30- 1,168.00","1,273.72","44,781,000- 81,420","570,384.55","14,716,350","1,632,950: 12.48%"
2,TATASTEEL,28-Mar-24,157.80,5.20,3.41%,159.85- 153.00,157.19,"262,174,000- 47,668","412,111.31","234,762,000","23,589,500: 11.17%"
3,TATAMOTORS,28-Mar-24,"1,039.50",15.90,1.55%,"1,048.90- 1,016.50","1,036.78","39,627,825- 27,809","410,853.36","57,753,825","-1,239,750: -2.10%"
4,BEL,28-Mar-24,215.90,4.40,2.08%,217.75- 212.40,215.26,"115,385,100- 20,243","248,377.97","138,441,600","-2,707,500: -1.92%"
5,HDFCBANK,28-Mar-24,"1,449.30",1.30,0.09%,"1,455.10- 1,445.25","1,449.43","16,569,850- 30,127","240,168.38","180,727,800","-4,179,450: -2.26%"
6,SBIN,28-Mar-24,788.10,1.35,0.17%,795.65- 785.00,790.79,"28,824,000- 19,216","227,937.31","83,557,500","-4,179,000: -4.76%"
7,RELIANCE,28-Mar-24,"2,967.65",-51.50,-1.71%,"3,017.90- 2,962.95","2,985.21","7,338,000- 29,352","219,054.71","39,454,750","456,250: 1.17%"
8,BAJAJ-AUTO,28-Mar-24,"8,874.00",238.95,2.77%,"8,968.80- 8,523.10","8,804.69","2,211,000- 17,688","194,671.70","2,287,625","99,750: 4.56%"
9,SAIL,28-Mar-24,139.50,-0.35,-0.25%,144.10- 139.00,141.42,"131,312,000- 16,414","185,701.43","147,576,000","8,744,000: 6.30%"
